In [1]:
%pip install -U -q google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 6.3 MB/s eta 0:00:00


# Procesamiento de Lenguaje Natural con Google Gemini API
Este script demuestra diversas aplicaciones de Procesamiento de Lenguaje Natural (PLN) utilizando el modelo Gemini de Google a través de la librería `google-generativeai`.

In [2]:
from google.colab import userdata
import os

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [3]:
from google import genai
from google.genai import types

cliente = genai.Client(api_key=GOOGLE_API_KEY)

# Definición del texto de entrada

In [4]:
# Texto base que utilizaremos para todos los ejemplos.

text_to_process = """Estimado Amazon, la semana pasada pedí una figura de acción de Optimus Prime
en su tienda en línea en Alemania. Desafortunadamente, cuando abrí el paquete,
descubrí con horror que me habían enviado una figura de acción de Megatron
en su lugar. Como enemigo de toda la vida de los Decepticons, espero que pueda
entender mi dilema. Para resolver el problema, exijo un cambio de Megatron por
la figura de Optimus Prime que pedí. Adjunto copias de mis registros relativos
a esta compra. Espero tener noticias suyas pronto. Atentamente, Bumblebee."""

print("\nTexto de entrada definido.")


Texto de entrada definido.


In [5]:
MODEL_ID = "gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-05-20","gemini-2.5-pro-preview-05-06"] {"allow-input":true, isTemplate: true}

## 1. Sumarizacion

In [6]:
pregunta = f"""Sumariza el siguiente texto en dos oraciones de rapida lectura

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[pregunta] # Pasa la pregunta como contenido
)
print(respuesta.text)

Bumblebee le reclama a Amazon que en lugar de su figura de Optimus Prime, recibió una de Megatron. Exige que le cambien la figura de Megatron por la de Optimus Prime que había ordenado.



##2. Clasificación de Sentimiento

In [7]:
pregunta = f"""Clasificá el siguiente texto como positivo, negativo o neutral y explicá por qué:

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[pregunta] # Pasa la pregunta como contenido
)
print(respuesta.text)

El texto se clasifica como **negativo**.

**Explicación:**

Aunque el tono general es educado y formal (usa "Estimado Amazon", "Atentamente", etc.), el contenido expresa **insatisfacción y frustración**. Los siguientes elementos indican una connotación negativa:

*   **"Desafortunadamente"**: Indica un evento no deseado.
*   **"Con horror"**: Expresa una fuerte reacción negativa ante el error.
*   **"Dilema"**: Implica una situación problemática.
*   **La necesidad de una "exigencia"**: Señala un problema que requiere resolución.
*   **El contraste entre el pedido y la recepción**: El cliente pidió Optimus Prime y recibió a Megatron, su enemigo, lo que claramente es un error y una fuente de decepción.

En resumen, si bien el lenguaje es cortés, el texto transmite una experiencia negativa debido a un error en el pedido y la necesidad de corregirlo.



##3. Reconocimiento de Entidades Nombradas (NER)

In [8]:
prompt = f"""Extraé todas las entidades nombradas del siguiente texto (personas, organizaciones, lugares, objetos) y clasificálas:

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)

Aquí está la extracción de entidades nombradas del texto proporcionado, clasificadas por tipo:

*   **Organizaciones:**
    *   Amazon
    *   Decepticons

*   **Personas:**
    *   Bumblebee

*   **Lugares:**
    *   Alemania

*   **Objetos:**
    *   Optimus Prime (figura de acción)
    *   Megatron (figura de acción)


##4. Respuesta a preguntas (Question Answering)

In [9]:
pregunta = "¿Qué producto recibió el cliente?"
contexto = text_to_process

prompt = f"""Respondé la siguiente pregunta basada en el texto:

Texto: {contexto}
Pregunta: {pregunta}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[contexto, pregunta] # Pasa la pregunta como contenido
)
print(respuesta.text)


El cliente recibió una figura de acción de Megatron.



##5. Resumen automático

In [10]:
prompt = f"""Resumí el siguiente texto en no más de 3 líneas:

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)


Bumblebee recibió por error una figura de acción de Megatron en lugar de la de Optimus Prime que había pedido a Amazon Alemania. Exige un cambio por el artículo correcto y adjunta la documentación de su pedido. Espera una pronta respuesta.



##6. Traducción (Español a Inglés)

In [11]:
prompt = f"""Traducí al inglés este texto:

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)


Okay, here's the English translation:

Dear Amazon,

Last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent a Megatron action figure instead. As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve this issue, I demand an exchange of the Megatron figure for the Optimus Prime figure that I ordered. I have attached copies of my records pertaining to this purchase. I look forward to hearing from you soon.

Sincerely,

Bumblebee



##7. Generación de respuesta (como atención al cliente)

In [12]:
respuesta_inicial = "Estimado cliente, lamentamos mucho lo ocurrido con su pedido. "

prompt = f"""{text_to_process}

Redactá una respuesta del servicio de atención al cliente que comience así:

"{respuesta_inicial}"

Cuya extension no supere las 4 lineas.
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)

Estimado cliente, lamentamos mucho lo ocurrido con su pedido. Entendemos su frustración por recibir a Megatron en lugar de Optimus Prime. Le pedimos disculpas por el error y procesaremos el cambio solicitado de inmediato. Por favor, siga las instrucciones adjuntas para la devolución y le enviaremos su Optimus Prime lo antes posible.



##8. Clasificación Zero-Shot (sin entrenamiento previo)

In [14]:
etiquetas = ["queja", "elogio", "consulta", "pedido", "agradecimiento"]

prompt = f"""Clasificá el siguiente texto en una de estas categorías: {', '.join(etiquetas)}. Justificá tu elección.

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)


**Clasificación: Queja**

**Justificación:**

El texto presenta los siguientes elementos que indican una queja:

*   **Identificación de un problema:** El autor expresa su insatisfacción porque recibió un producto incorrecto (Megatron en lugar de Optimus Prime).
*   **Expresión de disgusto:** El autor utiliza palabras como "horror" para transmitir su frustración por el error.
*   **Solicitud de una solución:** El autor "exige" un cambio del producto incorrecto por el producto que había ordenado.
*   **Referencia a la transacción:** El autor menciona el pedido en la tienda en línea y adjunta copias de los registros de la compra.

Todos estos elementos son característicos de una queja formal dirigida a la empresa Amazon.


## EJERCICIO

Escribí un texto corto sobre una experiencia personal en un transporte público en Buenos Aires.

Luego, generá:

- Un resumen.
- Una clasificación de sentimiento.
- Una lista de entidades nombradas.